<a href="https://colab.research.google.com/github/KeqingW44448/api/blob/main/RSM8421/4_1_Introduction_to_CNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network Example in Keras

This notebook shows an example of how to use CNNs in Keras.

## Load Libraries

We're using the same Keras libraries as before, except adding the `Conv2D`, `MaxPooling2D` and `Flatten` layers.

In [1]:
from tensorflow import keras
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

## Parameters

We define some parameters for convenience so that we can adjust them all in one place.

In [2]:
batch_size = 100
num_classes = 10
epochs = 10

# input image dimensions
img_rows, img_cols = 28, 28

## Load MNIST Data

We're going to use the same handwritten digit dataset as before.  Notice we have to "reshape" it so that we retain the length x width x channels dimensions of the dataset.  When we were only considered feed-forward networks, we simply flattened these dimensions, but since we want to utilize them, we must retain the shape.

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.
x_test /= 255.

# Convert class vectors to one-hot encoding
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Depending on the implementation, the underlying libraries might want the image
# dimensions in different orders, check for it and reshape
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

# For this demonstration, select 10k random samples for training
index = np.arange(x_train.shape[0])
np.random.shuffle(index)
index = index[:60000]
x_train, y_train = x_train[index], y_train[index]

print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
x_train shape: (60000, 28, 28, 1)
y_train shape: (60000, 10)
60000 train samples
10000 test samples


## Model

Here we're building a simple CNN.  Notice it follows the same `Sequential` API as feed-forward networks.  Simply stack a bunch of layers together.  In this case, we're using CNN layers, followed by a MaxPool layer, followed by an FC/softmax layer.  This is a typical architecture for CNN image classifiers.

Notes:

* The `filters`, `kernel_size`, `strides`, and `padding` parameters are used in the `Conv2D` layers, which correspond to the parameters for CNNs that we've learned.
* A `MaxPooling2D` layer is used to downsample the size of the CNN layers.
* `Flatten` layer must be used after the `Conv2D/MaxPooling2D` layers to reshape the output tensor so that the `Dense` layer can use it.
* Inspect the output of `model.summary()`, which provides valuable information about the dimensions of the CNN layers and can be helpful for debugging issues.

In [4]:
model = Sequential()
model.add(Conv2D(16, kernel_size=(3, 3),
                 strides=1, padding='valid',
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(16, kernel_size=(3, 3),
                 strides=2, padding='valid',
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=0.01),
              metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 16)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 12, 12, 16)     │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 6, 6, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 6, 6, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 576)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 40,058 (156.48 KB)

 Trainable params: 40,058 (156.48 KB)

 Non-trainable params: 0 (0.00 B)

## Training

Training uses the same API as our feed-forward networks.

In [5]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          verbose=0,
          callbacks=[])

## Evaluation

Finally, we can evaluate our progress on the test set as usual.

In [6]:
score = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.04712606221437454
Test accuracy: 0.9872999787330627
